教學: https://www.w3schools.com/python/python_mysql_getstarted.asp

請先安裝套件:

 `> pip install mysql-connector-python`

In [1]:
import mysql.connector
from mysql.connector import errorcode

In [2]:
# init variables
PRIMARY_KEY = 'name'

AWS RDS資料庫連線所需資訊(已過期，請另外新建)
* 名稱: rds-mysql-test
* 地址: rds-mysql-test.ccb6n449afg7.ap-northeast-1.rds.amazonaws.com
* 埠: 3306
* 帳號: admin
* 密碼: `(AWS自動產生)`

In [4]:
# 連線&錯誤回應
try:
    db = mysql.connector.connect(
        host='rds-mysql-test.ccb6n449afg7.ap-northeast-1.rds.amazonaws.com',
        user='admin',
        password='',  # AWS自動產生
        database='rds_mysql'  # 連線到特定database
    )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('資料庫連線失敗：帳號或密碼錯誤')
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print('資料庫不存在')
else:
    print(db)

mycursor = db.cursor()

In [ ]:
# 建立DB
mycursor = db.cursor()
mycursor.execute('CREATE DATABASE 資料庫名稱')

In [5]:
# 確認DB存在
mycursor = db.cursor()
mycursor.execute('SHOW DATABASES')
for item in mycursor:
    print(item)

('information_schema',)
('mysql',)
('performance_schema',)
('rds_mysql',)
('sys',)


In [ ]:
# 建立table
mycursor = db.cursor()
mycursor.execute('CREATE TABLE students (name VARCHAR(100), city VARCHAR(50), addr VARCHAR(200), pin VARCHAR(10))')

In [14]:
# 確認table存在
mycursor = db.cursor()
mycursor.execute('SHOW TABLES')
for item in mycursor:
    print(item)

('students',)


In [35]:
# CREATE(用預設語法)
mycursor = db.cursor()
sql = 'INSERT INTO students (name, city, addr, pin) VALUES (%s, %s, %s, %s)'
val = ('Garfield', 'Taipei', 'No.1, Beping E.St.', 'test')
mycursor.execute(sql, val)
db.commit()
# 計算資料
print(mycursor.rowcount, '新增資料成功')

1 新增資料成功


In [ ]:
# CREATE(資料放在字典再匯入)
mycursor = db.cursor()
input = {
    'name': 'Will',
    'pin': 'dict'
}

# Dict:input轉sql語法
newColumn = tuple(input.keys())
newPlaceholder = tuple(['%s'] * len(newColumn))
sql = f'INSERT INTO students {newColumn} VALUES {newPlaceholder}'
parsedSql = sql.replace("'", "")
newValue = tuple(input.values())

# 執行SQL
mycursor.execute(parsedSql, newValue)
db.commit()
print(mycursor.rowcount, '新增資料成功')

In [6]:
# READ(所有資料)
mycursor = db.cursor()
mycursor.execute('SELECT * FROM students')
for item in mycursor:
    print(item)
    print(type(mycursor))

(3, 'John', 'New York', 'Highway 21', 'ipynb')
<class 'mysql.connector.cursor_cext.CMySQLCursor'>
(4, 'John2', 'New York', 'Highway 21', 'ipynb2')
<class 'mysql.connector.cursor_cext.CMySQLCursor'>
(5, 'Garfield', 'Taipei', 'No.1, Beping E.St.', 'test')
<class 'mysql.connector.cursor_cext.CMySQLCursor'>
(6, 'Will', None, None, 'dict')
<class 'mysql.connector.cursor_cext.CMySQLCursor'>


In [38]:
# READ(特定一筆資料)
mycursor = db.cursor()
sql = f'SELECT * FROM students WHERE {PRIMARY_KEY} = %s'
val = ('Garfield', )
mycursor.execute(sql, val)
for item in mycursor:
    print(item)

(5, 'Garfield', 'Taipei', 'No.1, Beping E.St.', 'test')


In [22]:
# UPDATE(預設語法)
sql = 'UPDATE students SET pin = %s WHERE name = %s'
val = ('ipynb', 'John')
mycursor.execute(sql, val)
db.commit()
print(mycursor.rowcount, '更新資料成功')

1 新增資料成功


In [89]:
# UPDATE(資料放在字典再匯入)
mycursor = db.cursor()
input = {
    'name': 'Will',
    'addr': '1355 Vincent',
    'pin': 'dict'
}

# 抽出主key-value
primaryValue = input[PRIMARY_KEY]
parsedInput = input.copy()
del parsedInput[PRIMARY_KEY]

# Dict:parsedInput轉sql語法
newColumnAndPlaceholder = ''
for key in parsedInput.keys():
    newColumnAndPlaceholder += key + ' = %s, '
sql = f'UPDATE students SET {newColumnAndPlaceholder}WHERE {PRIMARY_KEY} = {primaryValue}'
parsedSql = sql.replace("'", "")
newValue = tuple(parsedInput.values())
print(parsedSql, '\n', newValue)

# 執行sql
# mycursor.execute(parsedSql, newValue)
# db.commit()
# print(mycursor.rowcount, '更新資料成功')

UPDATE students SET addr = %s, pin = %s, WHERE name = Will 
 ('1355 Vincent', 'dict')


In [33]:
# DELETE(慎用)
mycursor = db.cursor()
sql = f'DELETE FROM students WHERE {PRIMARY_KEY} = %s'
val = ('Garfield', )
mycursor.execute(sql, val)
db.commit()
print(mycursor.rowcount, '刪除資料成功')

2 刪除資料成功


In [8]:
# 關閉連線，資料庫操作完一定要執行
db.close()